In [1]:
import pandas as pd
import os
import email
# import tensorflow_text as text

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [3]:
# import tensorflow as tf

recuperation de tous les chemins des 10819 mails

In [2]:
path = "data/archives-EGC/"

email_paths = []
for root, dirs, files in os.walk(path):
    for file in files:
        if not file.startswith('.'):
            email_path = os.path.join(root, file)
            email_paths.append(email_path)

Fonction pour lire le contenu d'un email a partir du chemin

In [4]:
def get_email(email_path):
    with open(email_path, "r", encoding = "ISO-8859-1") as mail: # only returns a file object
        email_content = mail.read() # read the object
        return email_content

## Lire UN fichier

In [78]:
# Search one mail from given path
path = "data/archives-EGC/polytech_liste-egc_archives-2006-2010/polytech_liste-egc_2008-04/10"
msg = get_email(path)
msg = email.message_from_string(msg)
mails = []

for part in msg.walk():
    payload = part.get_payload(decode=True)
    if payload is None:
        continue
    try:
        body = payload.decode().decode(part.get_content_charset())
    except:
        body = payload.decode("ISO-8859-1")
        break
# body

In [72]:
# Search one mail from given index
path = email_paths[0]
print(path)

msg = get_email(path)
msg = email.message_from_string(msg)
for part in msg.walk():
        body = part.get_payload(decode=True).decode(part.get_content_charset())

data/archives-EGC/polytech_liste-egc_archives-2006-2010/polytech_liste-egc_2008-04/20


In [74]:
path = email_paths[5]
with open(path, "r", encoding = "ISO-8859-1") as mail:
       mail = mail.read()

'\tReturn-Path: <Omar.Boussaid@univ-lyon2.fr>\nX-Original-To: polytech_liste-egc@sympa.univ-nantes.fr\nDelivered-To: polytech_liste-egc@sympa.univ-nantes.fr\nReceived: from BounceSmtp1.univ-nantes.fr (BounceSMTP1.univ-nantes.prive [172.20.12.66])\n\tby Sympa.univ-nantes.fr (Postfix) with ESMTP id 1104318B7289\n\tfor <polytech_liste-egc@sympa.univ-nantes.fr>; Tue, 22 Apr 2008 15:43:33 +0200 (CEST)\nReceived: from MX2.univ-nantes.fr (MX2.univ-nantes.fr [193.52.101.136])\n\tby BounceSmtp1.univ-nantes.fr (Postfix) with ESMTP id 3E63DC3BD\n\tfor <polytech_liste-egc@sympa.univ-nantes.fr>; Tue, 22 Apr 2008 15:45:23 +0200 (CEST)\nReceived: from localhost (debian [127.0.0.1])\n\tby MX2.univ-nantes.fr (Postfix) with ESMTP id 8827B828B00\n\tfor <polytech_liste-egc@sympa.univ-nantes.fr>; Tue, 22 Apr 2008 15:46:40 +0200 (CEST)\nReceived: from MX2.univ-nantes.fr ([193.52.101.136])\n\tby localhost (MX2.univ-nantes.fr [193.52.101.136]) (amavisd-new, port 10024)\n\twith LMTP id 28267-01-38\n\tfor <poly

In [ ]:
substring = part.get_payload(decode=True)[787:787 + 10]
substring

# Pre-traitement

Lecture de tous les mails

In [5]:
mails = []
for path in email_paths:
    msg = get_email(path)
    msg = email.message_from_string(msg)
    for part in msg.walk():
        payload = part.get_payload(decode=True)
        if payload is None:
            continue
        try:
            body = payload.decode().decode(part.get_content_charset()) # catch any decoding error to decode in ISO-8859-1
        except:
            body = payload.decode("ISO-8859-1")
            # print(path)
            break # to avoid only getting the last body of a mail. Il peut y avoir du html a la fin donc on prend just ele premier body qui est du texte
    mails.append(body)
len(mails)

10819

# Mails multilangues

In [7]:
print(mails[0])

Bonjour à tous,

Le 23 Mai 2008 une journée nationale sur l'apprentissage aura lieu à 
Paris :

Apprentissage Artificiel : la carte, le territoire et l'horizon.
                 en l'honneur d'Yves Kodratoff

La matinée illustrera les nombreuses applications de l'apprentissage
ayant émergé dans les dernières années, au travers d'exposés courts
présentés par des experts académiques ou industriels.

L'après midi offrira plusieurs perspectives sur les priorités de la
prochaine décennie, avec les exposés invités de Gheorghe Tecuci
(George Mason University), Attilio Giordana (U. d'Alessandria), et
Stefan Wrobel (Fraunhofer Institute) - situées par rapport aux
priorités des années 90.

L'après midi continuera par un débat, discutant les référents en pleine 
évolution de l'apprentissage : logique, sciences cognitives,
statistiques, et optimisation.

La journée se finira par un cocktail, qui nous sera une occasion 
chaleureuse de lever nos verres en l'honneur d'Yves Kodratoff - avec 
notre rec

In [11]:
type(mails[0])

str

In [13]:
from langdetect import detect_langs

def split_text_by_language(text):
    paragraphs = text.split("\n\n")  # Split text into paragraphs
    language_sections = {'unknown': []}

    for paragraph in paragraphs:
        
        # Detect language for each paragraph
        language_probabilities = detect_langs(paragraph)
        
        # Get the most probable language
        most_probable_language = max(language_probabilities, key=lambda x: x.prob)
        
        # Group paragraphs by language
        language_code = most_probable_language.lang
        
        if language_code not in language_sections:
            language_sections[language_code] = []
        language_sections[language_code].append(paragraph)

    return language_sections

text = mails[0]

language_sections = split_text_by_language(text)

for language, paragraphs in language_sections.items():
    print(f"Language: {language}")
    print("\n".join(paragraphs))
    print("---------")


LangDetectException: No features in text.

In [12]:
from langdetect import detect_langs

def split_text_by_language(text):
    paragraphs = text.split("\n\n")  # Split text into paragraphs
    language_sections = {'unknown': []}

    for paragraph in paragraphs:
        
        # Detect language for each paragraph
        language_probabilities = detect_langs(paragraph)
        
        # Get the most probable language
        most_probable_language = max(language_probabilities, key=lambda x: x.prob)
        
        # Group paragraphs by language
        language_code = most_probable_language.lang
        
        if language_code not in language_sections:
            language_sections[language_code] = []
        language_sections[language_code].append(paragraph)

    return language_sections

text = """
Bonjour! Hello!
Comment ça va? I'm fine, thank you.
Et toi?

This is an example of English text.
Here is another sentence.

Ceci est un exemple de texte en français.
Et voici une autre phrase.
"""

language_sections = split_text_by_language(text)

for language, paragraphs in language_sections.items():
    print(f"Language: {language}")
    print("\n".join(paragraphs))
    print("---------")


LangDetectException: No features in text.

# Creation des Embeddings

In [80]:
preprocess_url = "https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3"
encoder_url = "https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4"

In [81]:
bert_preprocess_model = hub.KerasLayer(preprocess_url)

NameError: name 'hub' is not defined

In [ ]:
text_test = ['Il est vers la porte', 'Je marche vers lui', 'Ce vers est lent']
text_preprocessed = bert_preprocess_model(text_test)
text_preprocessed.keys()

In [ ]:
bert_model = hub.KerasLayer(encoder_url)
bert_results = bert_model(text_preprocessed)
bert_results.keys()

# Recupération vecteurs les plus proches

# Classification